In [9]:
# wir benötigen wieder ein paar python imports...
import sys
from os import environ
import os
import time
import uuid
import logging
sys.path.append(os.path.abspath(".."))


from sparql_builder import *
from connector_client import ConnectorClient
from connector.messages.datamodel_utils import SubscriptionRegisterRequest, SubscriptionUnregisterRequest # generic messages within the connector_base repo

client = ConnectorClient(bootstrap_servers=[environ.get("KAFKA_BROKER")])

# graphdb queries, um den richtigen connector zu identifizieren
await client.load_connector_config()
client.return_connectors()

dict_keys(['opcua_dev_connector'])

In [10]:
# wechsel an den ausgewählten, richtigen connector
await client.switch_connector('opcua_dev_connector')

In [11]:
output = await client.list_instances(class_uri="df:DataPoint")
output

,instance
0,9684034d-9b26-441e-8971-39e4d20000ea
1,00796da8-8876-4b02-af1a-ecf1a963c39a
2,3eb2e3e5-1605-4d46-9c42-2ca945160ea0
3,64812ec3-74f5-4f5d-bc81-044ddeb266cb
4,5701a9f5-a059-4f75-99a9-8904843029aa
5,09f46ad3-6dcf-4d4b-999f-766e8549401a
6,6b7a488a-0c2b-4ad3-9172-ffdf24246ff8
7,4a3c4977-a8f5-4604-b441-822a115707fc
8,54c4b372-7fa8-4ecb-8971-6e5978eaacb3
9,c6b2f5b7-55bf-45bb-9cb0-3f8a3b27fc2a


In [12]:
datapoint_uri = f"http://stephantrattnig.org/instances#{output.iloc[0]["instance"]}"
output = await client.resolve_and_subscribe(datapoint_uri=datapoint_uri)
output

({'version': '1.0.0',
  'payload': {'type': 'CommandResponse',
   'base_payload': {'type': 'SubscriptionRegisterResponse',
    'device_origin': 'opcua_dev_connector.ServiceModuleOPCUA',
    'response': 'Subscription successfully'}},
  'type': 'ResponseMessage',
  'message_id': UUID('6390fd8e-486a-4d04-ac74-c76ea7995623'),
  'correlation_id': UUID('eb3ab951-d9ae-4500-ad8d-d5a01b584ea1'),
  'timestamp': datetime.datetime(2025, 8, 3, 12, 40, 14, 668542),
  'status_code': 'Valid',
  'service_id': 'opcua_dev_connector'},
 {'version': '1.0.0',
  'payload': {'type': 'CommandResponse',
   'base_payload': {'type': 'SubscriptionRegisterResponse',
    'device_origin': 'opcua_dev_connector.MetadataModuleOPCUA',
    'response': 'Subscription registered in the knowledge graph'}},
  'type': 'ResponseMessage',
  'message_id': UUID('adb7ba93-dd85-4d1d-9e9a-19999726d3eb'),
  'correlation_id': UUID('eb3ab951-d9ae-4500-ad8d-d5a01b584ea1'),
  'timestamp': datetime.datetime(2025, 8, 3, 12, 40, 14, 696753),


In [5]:
# manually subscribe (old way by manually ingesting the payload)
subscription_id = uuid.uuid4()

example_subscribe_payload = SubscriptionRegisterRequest(
    datapoint_identifier=["ns=2;i=59"],
    device_origin="client",
    subscription_identifier=subscription_id
)

await client.send_subscribe_and_await_response(base_payload=example_subscribe_payload) # hint: for now this is wrong. Adapt later

#example_unsubscribe_payload = SubscriptionUnregisterRequest(
#    subscription_identifier = uuid.uuid4(),
#    device_origin="client"
#)

({'version': '1.0.0',
  'payload': {'type': 'CommandResponse',
   'base_payload': {'type': 'SubscriptionRegisterResponse',
    'device_origin': 'opcua_dev_connector.ServiceModuleOPCUA',
    'response': 'Subscription successfully'}},
  'type': 'ResponseMessage',
  'message_id': UUID('07ebf384-9141-47a8-9f9b-4b917344fa3f'),
  'correlation_id': UUID('ad558a6d-e821-43f3-bfbc-67af5f82fd9c'),
  'timestamp': datetime.datetime(2025, 8, 1, 13, 32, 28, 717395),
  'status_code': 'Valid',
  'service_id': 'opcua_dev_connector'},
 {'version': '1.0.0',
  'payload': {'type': 'CommandResponse',
   'base_payload': {'type': 'SubscriptionRegisterResponse',
    'device_origin': 'opcua_dev_connector.MetadataModuleOPCUA',
    'response': 'Subscription registered in the knowledge graph'}},
  'type': 'ResponseMessage',
  'message_id': UUID('215d9725-0fec-437e-bf30-a95fce72c714'),
  'correlation_id': UUID('ad558a6d-e821-43f3-bfbc-67af5f82fd9c'),
  'timestamp': datetime.datetime(2025, 8, 1, 13, 32, 28, 738864),


In [30]:
# generally: list all subscriptions and their states 
await client.list_instances(class_uri="df:Subscription", optional_props=["rdfs:label", "df:hasSubscriptionState", "df:subscribesToDataPoint"])

,instance,label,hasSubscriptionState,subscribesToDataPoint
0,5ce9699c-d0ee-4192-a65f-8dfc7f0fcd4e,Subscription,Active,a8f37154-69bb-477a-8ea9-6d10f726d4a1
1,5ce9699c-d0ee-4192-a65f-8dfc7f0fcd4e,Subscription,Active,1fc7fbba-1136-471e-89b4-48d7e971ebb7
2,0e11663a-ad30-46c1-b908-d7fa1cecfeb2,Subscription,Active,53c3c5e5-b0a2-4a29-b9b2-2536aa0c07a6
3,562257f5-7e98-40ca-877f-ff52757cbfd5,Subscription,Active,53c3c5e5-b0a2-4a29-b9b2-2536aa0c07a6
4,16348229-5a2d-476b-ba45-0ce5b99725ca,Subscription,Active,53c3c5e5-b0a2-4a29-b9b2-2536aa0c07a6
5,644517fd-d752-4485-b2e1-a34adf479d5e,Subscription,Active,53c3c5e5-b0a2-4a29-b9b2-2536aa0c07a6
6,9aaf5676-5295-469b-9108-e21e014b8590,Subscription,Active,53c3c5e5-b0a2-4a29-b9b2-2536aa0c07a6
7,9cdddb78-2dc9-452e-b439-14a0ad226d4f,Subscription,Active,82e0b0d2-269a-411c-9fff-d324d55e3d02
8,9cdddb78-2dc9-452e-b439-14a0ad226d4f,Subscription,Active,fe5f3552-14e2-40d7-bdee-938dec8a84d7
9,9cdddb78-2dc9-452e-b439-14a0ad226d4f,Subscription,Active,492821d1-a5e0-4dc6-a77d-013a2f0d961c


### This is exmperimental Stuff
(might be broken)

In [8]:
# manually subscribe (old way by manually ingesting the payload)
subscription_id = uuid.uuid4()

example_subscribe_payload = SubscriptionRegisterRequest(
    datapoint_identifier=["ns=2;i=59"],
    device_origin="client",
    subscription_identifier=subscription_id
)

example_unsubscribe_payload = SubscriptionUnregisterRequest(
    subscription_identifier = uuid.uuid4(),
    device_origin="client"
)

await client.send_subscribe_and_await_response(base_payload=example_unsubscribe_payload)



Timeout occurred, but one response was received.


[MsgModel(root=ResponseMessage(version='1.0.0', payload=CommandResponse(type='CommandResponse', base_payload=GenericErrorResponse(type='GenericErrorResponse', device_origin='opcua_dev_connector.ServiceModuleOPCUA', error="Unknown ReadCommand: type='SubscribeCommand' base_payload=SubscriptionUnregisterRequest(type='SubscriptionUnregisterRequest', device_origin='client', subscription_identifier=UUID('2beab806-be61-422f-b1a2-0f67a12b19d4'))")), type='ResponseMessage', message_id=UUID('9e4536ac-d435-4b2c-a160-60a2ca7bab3c'), correlation_id=UUID('c640a50e-c901-4767-aedb-42049c455e35'), timestamp=datetime.datetime(2025, 8, 3, 12, 40, 2, 473891), status_code='Valid', service_id='opcua_dev_connector'))]


{'type': 'GenericErrorResponse',
 'device_origin': 'opcua_dev_connector.ServiceModuleOPCUA',
 'error': "Unknown ReadCommand: type='SubscribeCommand' base_payload=SubscriptionUnregisterRequest(type='SubscriptionUnregisterRequest', device_origin='client', subscription_identifier=UUID('2beab806-be61-422f-b1a2-0f67a12b19d4'))"}